In [1107]:
import pandas as pd
import numpy as np

import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

from sklearn.pipeline import Pipeline, FeatureUnion

import statsmodels.api as sm

In [1096]:
import pandas as pd
import numpy as np
import datetime as dt

class DataRetrieval():
    
    #def __init__(self):

    
    def get_calls_data(self, filepath='../data/Calls_Table_data.csv', delimiter='\t'):
        """Retrieves call data from filepath"""
        df = pd.read_csv(filepath, delimiter='\t', encoding='utf-16')
        df.drop_duplicates(inplace=True)
        df.reset_index(drop=True, inplace=True)        
        return df
    
    def get_weather_data(self, filepath='../data/historical_weather.csv'):
        """Retrieves weather data from filepath"""
        df = pd.read_csv(filepath)
        df['date'] = pd.to_datetime(df['DATE']).dt.date
        weather_hist = (df[['date', 'TMAX', 'PRCP', 'SNOW']]
                        .rename(columns={'PRCP':'precip', 'TMAX':'temp_max',
                                         'PRCP':'precip', 'SNOW':'snow'}))
        weather_hist['precip^2'] = weather_hist['precip']**2
        weather_hist['snow^2'] = weather_hist['snow']**2      
        return weather_hist
    
    def get_seahawks_schedule(self, filepath='../data/seahawks_schedule.csv'):
        """
        Retrieves Seahawks game schedule from filepath
        
        Will need to rewrite scraper used to retrieve this data
        """
        df_in = pd.read_csv(filepath)
        df_dropna = df_in.copy()[df_in['Opp'].notna()].reset_index(drop=True)
        df = df_dropna.copy()[df_dropna['Opp'] != 'Bye Week'].reset_index(drop=True)
        
        
        df['date'] = ''
        for i in range(len(df)):
            df['date'][i] = (f"{df.iloc[i]['Date']}, {df.iloc[i]['year']}")
            
        df['date'] = pd.to_datetime(df['date']).dt.date
        
        df['seahawks_game'] = ''
        for i in range(len(df)):
            if type(df.iloc[i]['Unnamed: 10']) == str:
                if df.iloc[i]['Week'] == 'SuperBowl':
                    df['seahawks_game'][i] = 'SuperBowl'
                elif df.iloc[i]['Week'] in ['Wild Card', 'Division','Conf. Champ.']:
                    df['seahawks_game'][i] = 'away_Playoffs'
                else:
                    df['seahawks_game'][i] = 'away_Regular'
            else:     
                if df.iloc[i]['Week'] == 'SuperBowl':
                    df['seahawks_game'][i] = 'SuperBowl'
                elif df.iloc[i]['Week'] in ['Wild Card', 'Division','Conf. Champ.']:
                    df['seahawks_game'][i] = 'home_Playoffs'
                else:
                    df['seahawks_game'][i] = 'home_Regular'
        return df[['date', 'seahawks_game']].copy()
    
    def get_huskies_schedule(self, filepath='../data/huskies_schedule.csv'):
        """
        Retrieves Huskies game schedule from filepath

        Will need to rewrite scraper used to retrieve this data
        """
        df_in = pd.read_csv(filepath)
        df = df_in.copy()
        df['date'] = pd.to_datetime(df['Date']).dt.date

        df['huskies_game'] = ''
        for i in range(len(df)):
            if type(df.iloc[i]['Unnamed: 6']) == str:
                df['huskies_game'][i] = 'away'
            else:
                df['huskies_game'][i] = 'home'
        return df[['date','huskies_game']].copy()

    def get_sounders_schedule(self, filepath='../data/sounders_schedule.csv'):
        """
        Retrieves Sounders FC game schedule from filepath

        Will write instructions for getting game data
        """
        df_in = pd.read_csv(filepath)
        df = df_in.copy()
        df['date'] = pd.to_datetime(df_in['date_dd_mm_yy']).dt.date
        
        df['sounders_game'] = ''
        for i in range(len(df)):
            if df['home_team'][i].split()[0] == 'Seattle':
                df['sounders_game'][i] = 'home'
            else:
                df['sounders_game'][i] = 'away'
        return df[['date','sounders_game']].drop_duplicates()



In [1183]:
import pandas as pd
import numpy as np
import datetime as dt


class CountCalls():
    """Counts calls by date either by city or neighborhood"""
    
    def __init__(self, how='city'):
        self.how = how
        self.X = None
        self.y = None
        
    def fit(self, X, y=None):
        self.X = X
        self.y = y       
        return self
    
    def transform(self):
        
        if self.how == 'city':
            df = self.X[['ORIG_TIME_QUEUED', 'EVENT']].copy()
            df['date'] = pd.to_datetime(df['ORIG_TIME_QUEUED']).dt.date
            df.drop('ORIG_TIME_QUEUED', axis=1, inplace=True)
            return df.groupby('date').count().rename(columns={'EVENT':'num_calls'}).reset_index()
        
        else:
            df = self.X[['NEIGHBORHOOD', 'ORIG_TIME_QUEUED', 'EVENT']].copy()
            df['date'] = pd.to_datetime(df['ORIG_TIME_QUEUED']).dt.date
            df.drop('ORIG_TIME_QUEUED', axis=1, inplace=True)
            counts = df.groupby(['NEIGHBORHOOD', 'date']).count().rename(columns=
                                                                         {'NEIGHBORHOOD':'neighborhood',
                                                                          'EVENT':'num_calls'}).reset_index()
            
            neighborhoods = list(counts['NEIGHBORHOOD'].unique())
            num_days = int(np.timedelta64((max(counts['date']) - min(counts['date'])), 'D')/np.timedelta64(1,'D'))+1
            start = pd.to_datetime(min(counts['date']))
            neighboor_arr = np.array([(neighborhoods*num_days)])
            neighboor_arr = neighboor_arr.flatten()
            dates = [(start + np.timedelta64(i,'D')) for i in range(num_days)]*len(neighborhoods)
            
            df2 = pd.DataFrame({"dt_time": dates})
            df2['date'] = df2["dt_time"].dt.date
            df2['neighborhood'] = neighboor_arr
            df3 = pd.merge(df2, counts, how='outer', left_on=['date','neighborhood'],
                           right_on=['date','NEIGHBORHOOD']).fillna(0)
            return df3[['date', 'neighborhood', 'num_calls']]
        
        

class FeaturizeCalls():
    """Clean incoming df to fit into model"""
    
    def __init__(self):
        self.X = None
        self.y = None
    
    def fit(self, X, y=None):
        self.X = X
        self.y = y
        return self

    
    def transform(self):
        """tranform and clean incoming training or test"""
    
        df = self.X.copy()
        num_days = int(np.timedelta64((max(df['date']) - min(df['date'])), 'D')/np.timedelta64(1,'D'))+1
        start = pd.to_datetime(min(df['date']))
        dates = [(start + np.timedelta64(i,'D')) for i in range(num_days)]

        seq = pd.DataFrame({'dt_time': dates, 'day_seq':np.arange(num_days)})
        seq['date'] = seq['dt_time'].dt.date

        df1 = df.join(seq.set_index('date'), on='date')

        df1['year'] = df1['dt_time'].dt.year
        df1['month'] = df1['dt_time'].dt.month
        df1['day'] = df1['dt_time'].dt.day
        df1['day_of_week'] = df1['dt_time'].dt.weekday
        df1['month_day'] = df1['dt_time'].dt.strftime('%m/%d')
        df1['month_weekday'] = df1['dt_time'].dt.strftime('%b_%a')
        df1['month'] = df1['dt_time'].dt.strftime('%m/%d')     
        return df1


class DateDummies():

    def __init__(self):
        self.X = None
        self.y = None
    
    def fit(self, X, y=None):
        # X is the featurized calls dataframe
        self.X = X 
        self.y = y
        return self

    def transform(self):
        day_features= ['01/06','01/14','02/02','02/07','03/10','03/20','03/26','04/01','04/08','04/17',
                       '04/18','05/06','05/09','05/10','05/21','05/31','06/04','06/05','06/13','06/28',
                       '06/30','07/09','07/16','07/20','07/23','07/29','08/01','08/20','08/24','08/28',
                       '09/08','09/11','09/14','09/30','10/02','10/17','10/26','11/01','11/15','11/16',
                       '11/23','11/27','12/04','12/12','12/19','12/21','12/23','12/29']
        spec_days = pd.DataFrame({'month_day':day_features, 'spec_day':day_features})
        df =self.X.join(spec_days.set_index('month_day'), on='month_day')

        date_dummies =  pd.get_dummies(df[['date', 'day', 'month_weekday', 'spec_day']].set_index('date'),
                                       columns=['day', 'month_weekday', 'spec_day']).reset_index().drop_duplicates()
        return df.join(date_dummies.set_index('date'), on='date').fillna(0)   
    

class HolidayDummies():
    
    def __init__(self):
        self.X = None
        self.y = None
    
    def fit(self, X, y=None):
        # X is a dictionary of Holidays
        self.X = X 
        self.y = y
        return self

    def transform(self):
        _holidays = []
        for date in sorted(self.X.keys()):
            _holidays.append([date, self.X[date]])
        _holidays = pd.DataFrame(_holidays, columns=['date', 'holiday'])
        return pd.get_dummies(_holidays.set_index('date')).reset_index()
    
    
class EventDummies():
    
    def __init__(self, event_dict=None):
        self.X = None
        self.y = None
        self.event_dict = event_dict

        defualt_events = ({'Pride Parade' : ['6/30/2019', '6/24/2018', '6/25/2017', '6/26/2016', '6/28/2015',
                                          '6/29/2014', '6/30/2013', '6/24/2012', '6/26/2011', '6/27/2010'],
                            'Seafair' : ['8/2/2019', '8/3/2019', '8/4/2019', '8/3/2018', '8/4/2018', '8/5/2018',
                                         '8/4/2017', '8/5/2017', '8/6/2017', '8/5/2016', '8/6/2016', '8/7/2016',
                                         '7/31/2015', '8/1/2015', '8/2/2015', '8/1/2014', '8/2/2014', '8/3/2014',
                                         '8/2/2013', '8/3/2013', '8/4/2013', '8/3/2012', '8/4/2012', '8/5/2012',
                                         '8/5/2011', '8/6/2011', '8/7/2011', '8/6/2010', '8/7/2010', '8/8/2010' ],
                            'Soltice Parade': ['6/30/2019', '6/16/2018', '6/17/2017', '6/18/2016', '6/20/2015',
                                               '6/21/2014', '6/22/2013', '6/16/2012', '6/18/2011', '6/19/2010'],
                            'Womens March' : ['1/19/2019', '1/20/2018', '1/21/2017'],})
        
        if self.event_dict == None:
            self.event_dict = defualt_events
        
    
    def fit(self, X=None, y=None):
        self.X = X 
        self.y = y
        return self
    
    def transform(self):
        _events = []
        for event in self.event_dict.keys():
            for day in self.event_dict[event]:
                _events.append([dt.datetime.strptime(day, '%m/%d/%Y'), event])
                
        _events = pd.DataFrame(_events, columns=['date','local_event'])
        _events['date'] = _events['date'].dt.date
        return pd.get_dummies(_events.set_index('date')).reset_index()
    
    
class MakeDummies():
    
    def __init__(self):
        self.X = None
        self.y = None
    
    def fit(self, X, y=None):
        # X is a dataframe of sporting events
        self.X = X 
        self.y = y
        return self

    def transform(self):
        return pd.get_dummies(self.X.set_index('date')).reset_index()    
        

class JoinDataFrames():
    
    def __init__(self, weather, us_holidays, islamic_holidays, 
                 jewish_holidays, events, seahawks, huskies, sounders):
        self.weather = weather
        self.date_dummies = date_dummies
        self.us_holidays = us_holidays
        self.islamic_holidays = islamic_holidays
        self.jewish_holidays = jewish_holidays
        self.events = events
        self.seahawks = seahawks
        self.huskies = huskies
        self.sounders = sounders
        self.X = None
        self.y = None

    def fit(self, X, y=None):
        self.X = X
        self.y = y
        return self
    
    def transform(self):
        df1 = self.join_dfs(self.X, self.weather)
        df2 = self.join_dfs(df1, self.us_holidays)
        df3 = self.join_dfs(df2, self.islamic_holidays)
        df4 = self.join_dfs(df3, self.jewish_holidays)
        df5 = self.join_dfs(df4, self.events)
        df6 = self.join_dfs(df5, self.seahawks)
        df7 = self.join_dfs(df6, self.huskies)
        df8 = self.join_dfs(df7, self.sounders)
        return df8.fillna(0)
            
    def join_dfs(self, df1, df2):
        return df1.join(df2.set_index('date'), on='date')



In [476]:
len(df2), len(neighboor_arr), len(dates)

(188505, 188505, 188505)

In [1074]:
import pandas as pd
import numpy as np
import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

class SeattleHolidays:
    
    
    class CustomHolidays(holidays.US):
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            # Populate the holiday list with the default US holidays
            holidays.US._populate(self, year)
            # Example: Add Ninja Turtle Day
            #self[dt.date(year, 7, 13)] = "Ninja Turtle Day"
            for year in range(start_year, end_year):
                # Add Valentine's day
                self[dt.date(year, 2, 14)] = "Valentines Day"
                # Add St Patricks Day
                self[dt.date(year, 3, 17)] = "St Patricks Day"
                # Add Easter
                self[holidays.easter(year=year)] = "Easter"
                # Add Good Friday
                self[holidays.easter(year=year)  -  dt.timedelta(days=2)] = "Good Friday"
                # Add May Da
                self[dt.date(year, 5, 1)] = "May Day"
                # Add Cinco De Mayo
                self[dt.date(year, 5, 5)] = "Cinco De Mayo"
                # Add Halloween
                self[dt.date(year, 10, 31)] = "Halloween"
                # Add Día de Muertos
                self[dt.date(year, 11, 2)] = "Halloween"
                # Add Christmas Eve
                self[dt.date(year, 12, 24)] = "Christmas Eve"
                # Add New Years Eve
                self[dt.date(year, 12, 31)] = "New Years Eve"
                # Add Chinese New Year
                chinese = HongKong()
                for date, label in chinese.get_chinese_new_year(year):
                    self[date] = label


    class IslamicHolidays(holidays.HolidayBase):
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            qatar_holidays = Qatar()
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                days = qatar_holidays.get_calendar_holidays(year)
                # Add Ramadan
                for i in range(1, len(days)):
                    if qatar_holidays.get_calendar_holidays(year)[i][1] == 'Start of ramadan':
                        for day in range(30):
                            self[qatar_holidays.get_calendar_holidays(year)[i][0] 
                                 + dt.timedelta(days=day-1)] = "Ramadan"
                    else:
                        self[qatar_holidays.get_calendar_holidays(2018)[1][0] 
                             - dt.timedelta(days=2)] = qatar_holidays.get_calendar_holidays(year)[i][1]


    class JewishHolidays(holidays.HolidayBase):
        def retrieve_data(self, filepath):
            df = pd.read_csv(filepath)
            return df

        def get_holidays(self, paths_list):
            df = self.retrieve_data(paths_list[0])
            for filepath in paths_list[1 : ]:
                cal = self.retrieve_data(filepath)
                df = pd.concat([df, cal])
            return df

        def _populate(self, year=2019, paths_list=['../data/hebcal_2010_usa.csv',
                                                   '../data/hebcal_2015_usa.csv', 
                                                   '../data/hebcal_2020_usa.csv',
                                                   '../data/hebcal_2025_usa.csv'],
                      start_year=2009, end_year=2030):
            hebcal = self.get_holidays(paths_list)
            hebcal.reset_index(drop=True, inplace=True)
            hebcal['date'] = pd.to_datetime(hebcal["Start Date"]).dt.date
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                for i in range(len(hebcal)):
                    self[hebcal['date'][i]] = hebcal['Subject'][i]


In [972]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('counter', CountCalls()),
    ('feturizer', FeaturizeCalls()),
    ('date_dummifier', DateDummies()),
    ('us_holidays', HolidayDummies()),
    ('islamic_holidays', HolidayDummies()),
    ('jewish_holidays', HolidayDummies()),
    ('event_dummifier', EventDummies()),
    ('seahawks_dummifier', MakeDummies()),
    ('huskies_dummifier', MakeDummies()),
    ('ssounders_dummifier', MakeDummies()),
    ('data_joiner', JoinDataFrames())
])


TypeError: __init__() missing 9 required positional arguments: 'weather', 'date_dummies', 'us_holidays', 'islamic_holidays', 'jewish_holidays', 'events', 'seahawks', 'huskies', and 'sounders'

In [1090]:
retriever = DataRetrieval()

In [1091]:
calls = retriever.get_calls_data()

In [1093]:
counter = CountCalls('city')

In [1094]:
counter.fit(calls)

In [1095]:
calls_xfrmd = counter.transform()

In [989]:
len(calls_xfrmd)

3195

In [990]:
featurizer = FeaturizeCalls()

In [991]:
featurizer.fit(calls_xfrmd)

In [992]:
calls_w_features = featurizer.transform()

In [1100]:
len(calls_w_features)

3195

In [1184]:
dt_dummifier = DateDummies()

In [1185]:
dt_dummifier.fit(calls_w_features)

In [1186]:
dt_dummies = dt_dummifier.transform()

In [1187]:
dt_dummies.columns

Index(['date', 'num_calls', 'dt_time', 'day_seq', 'year', 'month', 'day',
       'day_of_week', 'month_day', 'month_weekday',
       ...
       'spec_day_11/15', 'spec_day_11/16', 'spec_day_11/23', 'spec_day_11/27',
       'spec_day_12/04', 'spec_day_12/12', 'spec_day_12/19', 'spec_day_12/21',
       'spec_day_12/23', 'spec_day_12/29'],
      dtype='object', length=174)

In [1188]:
dt_dummies.columns.values[5:]

array(['month', 'day', 'day_of_week', 'month_day', 'month_weekday',
       'spec_day', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6',
       'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13',
       'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19',
       'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25',
       'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31',
       'month_weekday_Apr_Fri', 'month_weekday_Apr_Mon',
       'month_weekday_Apr_Sat', 'month_weekday_Apr_Sun',
       'month_weekday_Apr_Thu', 'month_weekday_Apr_Tue',
       'month_weekday_Apr_Wed', 'month_weekday_Aug_Fri',
       'month_weekday_Aug_Mon', 'month_weekday_Aug_Sat',
       'month_weekday_Aug_Sun', 'month_weekday_Aug_Thu',
       'month_weekday_Aug_Tue', 'month_weekday_Aug_Wed',
       'month_weekday_Dec_Fri', 'month_weekday_Dec_Mon',
       'month_weekday_Dec_Sat', 'month_weekday_Dec_Sun',
       'month_weekday_Dec_Thu', 'month_weekday_Dec_Tue',
       'month_

In [1149]:
test_df

array(['month_weekday_Sep_Wed'], dtype=object)

In [1123]:
test_df == day_features

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [1120]:
test_df[day_features]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [1076]:
day_features = ['month_day_01/06', 'month_day_01/14', 'month_day_02/02', 'month_day_02/07', 'month_day_03/10', 
                'month_day_03/20', 'month_day_03/26', 'month_day_04/01', 'month_day_04/08', 'month_day_04/17', 
                'month_day_04/18', 'month_day_05/06', 'month_day_05/09', 'month_day_05/10', 'month_day_05/21', 
                'month_day_05/31', 'month_day_06/04', 'month_day_06/05', 'month_day_06/13', 'month_day_06/28',
                'month_day_06/30', 'month_day_07/09', 'month_day_07/16', 'month_day_07/20', 'month_day_07/23',
                'month_day_07/29', 'month_day_08/01', 'month_day_08/20', 'month_day_08/24', 'month_day_08/28',
                'month_day_09/08', 'month_day_09/11', 'month_day_09/14', 'month_day_09/30', 'month_day_10/02',
                'month_day_10/17', 'month_day_10/26', 'month_day_11/01', 'month_day_11/15', 'month_day_11/16',
                'month_day_11/23', 'month_day_11/27', 'month_day_12/04', 'month_day_12/12', 'month_day_12/19',
                'month_day_12/21', 'month_day_12/23', 'month_day_12/29']


In [1175]:
day_features= ['01/06','01/14','02/02','02/07','03/10','03/20','03/26','04/01','04/08','04/17',
               '04/18','05/06','05/09','05/10','05/21','05/31','06/04','06/05','06/13','06/28',
               '06/30','07/09','07/16','07/20','07/23','07/29','08/01','08/20','08/24','08/28',
               '09/08','09/11','09/14','09/30','10/02','10/17','10/26','11/01','11/15','11/16',
               '11/23','11/27','12/04','12/12','12/19','12/21','12/23','12/29']

spec_days = pd.DataFrame({'month_day':day_features, 'spec_day':day_features})



In [1176]:
calls_w_features.join(spec_days.set_index('month_day'), on='month_day')

,date,num_calls,dt_time,day_seq,year,month,day,day_of_week,month_day,month_weekday,spec_day
0,2010-01-01,46,2010-01-01,0,2010,01/01,1,4,01/01,Jan_Fri,NaN
1,2010-01-02,19,2010-01-02,1,2010,01/02,2,5,01/02,Jan_Sat,NaN
2,2010-01-03,35,2010-01-03,2,2010,01/03,3,6,01/03,Jan_Sun,NaN
3,2010-01-04,28,2010-01-04,3,2010,01/04,4,0,01/04,Jan_Mon,NaN
4,2010-01-05,29,2010-01-05,4,2010,01/05,5,1,01/05,Jan_Tue,NaN
5,2010-01-06,34,2010-01-06,5,2010,01/06,6,2,01/06,Jan_Wed,01/06
6,2010-01-07,40,2010-01-07,6,2010,01/07,7,3,01/07,Jan_Thu,NaN
7,2010-01-08,31,2010-01-08,7,2010,01/08,8,4,01/08,Jan_Fri,NaN
8,2010-01-09,21,2010-01-09,8,2010,01/09,9,5,01/09,Jan_Sat,NaN
9,2010-01-10,33,2010-01-10,9,2010,01/10,10,6,01/10,Jan_Sun,NaN


In [1104]:
len(dt_dummies)

3195

In [997]:
weather = retriever.get_weather_data()

In [998]:
seahawks_schedule = retriever.get_seahawks_schedule()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [999]:
huskies_schedule = retriever.get_huskies_schedule()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1000]:
sounders_schedule = retriever.get_sounders_schedule()

In [1001]:
sports = SportsDummies()

In [1002]:
sports.fit(seahawks_schedule)

In [1003]:
seahawks = sports.transform()

In [1004]:
sports.fit(huskies_schedule)

In [1005]:
huskies = sports.transform()

In [1006]:
sports.fit(sounders_schedule)

In [1007]:
sounders = sports.transform()

In [1077]:
us_holiday_dict = SeattleHolidays.CustomHolidays()

In [1078]:
us_holiday_dict._populate()

In [1079]:
holidayier = HolidayDummies()

In [1080]:
holidayier.fit(us_holiday_dict)

In [1081]:
us_holidays = holidayier.transform()

In [1013]:
jewish_holiday_dict = SeattleHolidays.JewishHolidays()

In [1014]:
jewish_holiday_dict._populate()

In [1015]:
holidayier.fit(jewish_holiday_dict)

In [1016]:
jewish_holidays = holidayier.transform()

In [1017]:
islamic_holiday_dict = SeattleHolidays.IslamicHolidays()

In [1018]:
islamic_holiday_dict._populate()

/anaconda3/lib/python3.6/site-packages/calendra/core.py:760: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [1019]:
holidayier.fit(islamic_holiday_dict)

In [1020]:
islamic_holidays = holidayier.transform()

In [1021]:
islamic_holidays

,date,"holiday_Eid al-Adha, Eid al-Fitr",holiday_Ramadan
0,2009-08-21,0,1
1,2009-08-22,0,1
2,2009-08-23,0,1
3,2009-08-24,0,1
4,2009-08-25,0,1
5,2009-08-26,0,1
6,2009-08-27,0,1
7,2009-08-28,0,1
8,2009-08-29,0,1
9,2009-08-30,0,1


In [1022]:
event_dummies = EventDummies()

In [1023]:
event_dummies.fit()

In [1024]:
events = event_dummies.transform()

In [1083]:
joiner = JoinDataFrames(weather, dt_dummies, us_holidays, islamic_holidays, jewish_holidays,
              events, seahawks, huskies, sounders)

In [1084]:
joiner.fit(calls_w_features)

In [1085]:
calls_final = joiner.transform()

In [1086]:
len(calls_final)

3195

In [1087]:
calls_final

,date,num_calls,dt_time,day_seq,year,month,day,day_of_week,month_day,month_weekday,...,local_event_Womens March,seahawks_game_SuperBowl,seahawks_game_away_Playoffs,seahawks_game_away_Regular,seahawks_game_home_Playoffs,seahawks_game_home_Regular,huskies_game_away,huskies_game_home,sounders_game_away,sounders_game_home
0,2010-01-01,46,2010-01-01,0,2010,01/01,1,4,01/01,Jan_Fri,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-02,19,2010-01-02,1,2010,01/02,2,5,01/02,Jan_Sat,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2010-01-03,35,2010-01-03,2,2010,01/03,3,6,01/03,Jan_Sun,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-04,28,2010-01-04,3,2010,01/04,4,0,01/04,Jan_Mon,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-05,29,2010-01-05,4,2010,01/05,5,1,01/05,Jan_Tue,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,2010-01-06,34,2010-01-06,5,2010,01/06,6,2,01/06,Jan_Wed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2010-01-07,40,2010-01-07,6,2010,01/07,7,3,01/07,Jan_Thu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2010-01-08,31,2010-01-08,7,2010,01/08,8,4,01/08,Jan_Fri,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,2010-01-09,21,2010-01-09,8,2010,01/09,9,5,01/09,Jan_Sat,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,2010-01-10,33,2010-01-10,9,2010,01/10,10,6,01/10,Jan_Sun,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1088]:
X = calls_final.drop(labels= ['date', 'num_calls', 'dt_time','year',
                                                    'month', 'day','day_of_week', 'month_day',
                                                    'month_weekday'], axis=1)
y = calls_final['num_calls']

In [1057]:
linear_model = sm.OLS(y, X, hasconst=False).fit()

In [1072]:
linear_model.params[linear_model.params.values>3.5][114:]

month_day_01/01                         7.319032
month_day_01/06                         3.993864
month_day_01/14                         4.289055
month_day_02/02                         4.156440
month_day_02/07                         3.693606
month_day_03/10                         3.634155
month_day_03/20                         3.564627
month_day_03/26                         3.507804
month_day_04/01                         5.446000
month_day_04/08                         5.021672
month_day_04/17                         4.176021
month_day_04/18                         4.437234
month_day_05/06                         4.146278
month_day_05/09                         4.323508
month_day_05/10                         3.842301
month_day_05/21                         5.526962
month_day_05/31                         3.776559
month_day_06/04                         5.377650
month_day_06/05                         5.047857
month_day_06/13                         3.901144
month_day_06/28     

In [1058]:
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              num_calls   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.964
Method:                 Least Squares   F-statistic:                     161.5
Date:                Wed, 28 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:03:20   Log-Likelihood:                -9902.8
No. Observations:                3195   AIC:                         2.086e+04
Df Residuals:                    2668   BIC:                         2.406e+04
Df Model:                         527                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================================
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
day_seq                                                                -0.0019      0.000    -16.450      0.000      -0.002      -0.002
temp_max                                                                0.0884      0.017      5.074      0.000       0.054       0.123
precip                                                                  0.7965      1.032      0.772      0.440      -1.227       2.820
snow                                                                   -1.5304      1.402     -1.091      0.275      -4.280       1.219
precip^2                                                               -0.2274      0.796     -0.286      0.775      -1.788       1.333
snow^2                                                                  0.2576      0.279      0.922      0.357      -0.290       0.805
day_1                                                                  19.9887      0.900     22.216      0.000      18.224      21.753
day_2                                                                  18.9950      0.900     21.111      0.000      17.231      20.759
day_3                                                                  18.5789      0.905     20.520      0.000      16.804      20.354
day_4                                                                  19.6059      0.910     21.540      0.000      17.821      21.391
day_5                                                                  19.8006      0.916     21.619      0.000      18.005      21.597
day_6                                                                  18.8409      0.927     20.336      0.000      17.024      20.658
day_7                                                                  20.4088      0.914     22.329      0.000      18.617      22.201
day_8                                                                  18.9385      0.916     20.684      0.000      17.143      20.734
day_9                                                                  18.9279      0.907     20.866      0.000      17.149      20.707
day_10                                                                 18.5629      0.904     20.545      0.000      16.791      20.335
day_11                                                                 19.1263      0.907     21.098      0.000      17.349      20.904
day_12                                                                 17.8379      0.909     19.629      0.000      16.056      19.620
day_13                                                                 18.5712      0.911     20.392      0.000      16.785      20.357
day_14                                                                 18.4613      0.915  